# Machine Learning-101

### Predecir a partir de tus datos


En este notebook daremos una visión muy básica del Machine Learning con Python. Utilizaremos otra librería muy popular llamada **sklearn**

Preparado por: Andrés Leiva Araos -@ALeivaAraos

Este notebook está basado en el trabajo de Matt Harrison [*Learning the Pandas Library*](https://www.amazon.com/Learning-Pandas-Library-Munging-Analysis/dp/153359824X/ref=sr_1_3?ie=UTF8&qid=1505448275&sr=8-3&keywords=python+pandas).

In [ ]:
import pandas as pd
import matplotlib
%matplotlib inline 
import numpy as np
nyc = pd.read_csv('data/central-park-raw.csv', parse_dates=[0])

In [ ]:
nyc[:10].T

In [ ]:
# Aplicamos las transformaciones del notebook anterior

nyc.columns = [x.strip() for x in nyc.columns]
nyc.columns = [x.replace(' ', '_') for x in nyc.columns]
nyc.PrecipitationIn.replace("T", '0.001')
nyc.PrecipitationIn = pd.to_numeric(nyc.PrecipitationIn.replace("T", '0.001'))
nyc['Events'] = nyc.Events.fillna('')

In [ ]:
nyc[:10].T

# Machine Learning

Pandas tiene una integración natural con la librería **sklearn**.

Intentemos predecir la humedad (``y``) a partir de otras columnas (``X``).

Entrenaremos un modelo de [Random Forest](https://es.wikipedia.org/wiki/Random_forest) con una muestra de nuestra data, luego evaluaremos nuestro modelo con otra muestra distinta. A esta técnica se le denomina **cross-validation**.

[Como funcionan los Random Forrest](http://randomforest2013.blogspot.com/2013/05/randomforest-definicion-random-forests.html)

In [ ]:
# Cargamos las librerías de sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
nyc.columns

In [ ]:
# Seleccionamos la columna Mean_Humidity y la concatenamos consigo misma pero desfasada en un día
# Completamos la concatenación con la fecha EST al inicio
pd.concat([nyc.EST, nyc.Mean_Humidity, nyc.Mean_Humidity.shift(1)], axis=1)

In [ ]:
nyc.__len__()

In [ ]:
# Regresion - Intentemos predecir Mean_Humidity (y) a partir de otras columnas distintas (X)
# Obtenemos el set de datos para entrenamiento del modelo (X_train)
# Shift Humidity down to predict next day
X = nyc[[x for x in nyc.columns if 'Humid' not in x]]
y = nyc.Mean_Humidity.shift(1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X.columns

In [ ]:
# Imprime y comprara los largos de los datasets
# Quéporcentaje destinó a entrenamiento (train) y a prueba (test)?
# Tu código aquí...


In [ ]:
# Creamos el modelo 
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)



In [ ]:
# Necesitamos convertir las variables categóricas en cuantitativas
nyc_dummy = pd.get_dummies(nyc, columns=['Events'])
nyc_dummy.head()

In [ ]:
# Volvemos a intentarlo ...
X = nyc_dummy[[x for x in nyc_dummy.columns if 'Humid' not in x]]
y = nyc_dummy.Mean_Humidity.shift(1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X.T

In [ ]:
# Creamos el modelo (problemas con los floats)
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)



In [ ]:
# Eliminamos el timestamp
# Volvemos a intentar ...
def valid(col):
    return 'Humid' not in col and 'EST' not in col
X = nyc_dummy[[x for x in nyc_dummy.columns if valid(x)]]
y = nyc_dummy.Mean_Humidity.shift(1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Modelamos ...
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

In [ ]:
# Necesitamos eliminar los NA
# Usamos el método dropna()
def valid(col):
    return 'Humid' not in col and 'EST' not in col
nyc_dummy = nyc_dummy.dropna()
X = nyc_dummy[[x for x in nyc_dummy.columns if valid(x)]].iloc[1:]
y = nyc_dummy.Mean_Humidity.shift(1).dropna()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Modelamos ...
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

In [ ]:
# Medimos R2 (indicador de precisión del modelo 1 es perfecto 0 es terrible)
rf_model.score(X_test, y_test)

In [ ]:
type(y_test)

In [ ]:
pd.concat([pd.Series(rf_model.predict(X_test)), y_test.reset_index(
drop=True)], axis=1)

In [ ]:
sorted(zip(X.columns, rf_model.feature_importances_),
        key=lambda x: x[1], reverse=True)